In [1]:
# import the necessary libraries
import numpy as np
import cv2
import dlib

In [2]:
def landmark_detection(image):

    # laod an image then convert it to grey scale
    img = cv2.imread(image)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # detect the face then find the landmarks
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    faces = detector(img_gray)
    landmark_points_faces = []
    for face in faces:
        landmarks = predictor(img_gray, face)
        landmark_points = []
        for n in range(68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmark_points.append((int(x), int(y)))

            cv2.circle(img, (x, y), 3, (0, 0, 255), -1)
        landmark_points_faces.append(landmark_points)

    return landmark_points_faces, img


In [3]:
image = 'images/original_images/donald_trump_meme.jpg'
landmark_points, _ = landmark_detection(image)

In [4]:
landmark_points

[(151, 268),
 (159, 316),
 (170, 362),
 (182, 405),
 (196, 449),
 (220, 484),
 (260, 505),
 (308, 515),
 (362, 513),
 (417, 507),
 (467, 495),
 (510, 473),
 (537, 434),
 (549, 386),
 (553, 335),
 (554, 282),
 (554, 230),
 (172, 202),
 (189, 173),
 (222, 159),
 (259, 159),
 (293, 169),
 (349, 161),
 (383, 143),
 (423, 134),
 (464, 139),
 (493, 166),
 (324, 200),
 (324, 226),
 (323, 252),
 (323, 280),
 (297, 322),
 (315, 323),
 (334, 322),
 (354, 318),
 (374, 314),
 (212, 233),
 (229, 217),
 (251, 213),
 (278, 224),
 (255, 230),
 (232, 234),
 (387, 211),
 (407, 195),
 (431, 194),
 (456, 203),
 (434, 209),
 (410, 211),
 (294, 419),
 (303, 378),
 (321, 355),
 (338, 356),
 (354, 350),
 (382, 366),
 (406, 404),
 (389, 414),
 (365, 420),
 (347, 422),
 (329, 424),
 (310, 422),
 (306, 412),
 (324, 377),
 (340, 374),
 (357, 372),
 (394, 399),
 (360, 390),
 (343, 393),
 (326, 394)]

In [5]:
len(landmark_points)

68

In [ ]:
def applyConvexHull(points1_faces, points2_faces):
    
    
    hulls1 = []
    hulls2 = []
    
    faces1 = points1_faces.shape[0]
    faces2 = points2_faces.shape[0]
    for face in range(faces):
          # Find convex hull of the two landmark points
        hull1 = []
        hull2 = []
        
        points1 = points1_faces[face]
        points2 = points2_faces[face]

        hullIndex = cv2.convexHull(np.array(points2), returnPoints = False)

        for i in range(len(hullIndex)):
            hull1.append(points1[int(hullIndex[i])])
            hull2.append(points2[int(hullIndex[i])])
            
        hulls1.append(hull1)
        hulls2.append(hull2)

    return hull1, hull2

In [ ]:
def calculateDelaunayTriangles(image, points):

    img = cv2.imread(image)
    rect = (0, 0, img.shape[1], img.shape[0])

    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(points)
    triangleList = subdiv.getTriangleList()

    delaunayTri_indexes = []


    for t in triangleList:

        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        pts = [pt1, pt2, pt3]

        # draw the Triangulation
        cv2.line(img, pt1, pt2, (0, 0, 255), 2)
        cv2.line(img, pt2, pt3, (0, 0, 255), 2)
        cv2.line(img, pt1, pt3, (0, 0, 255), 2)

        index = []
        #Get face-points (from 68 face detector) by coordinates
        for i in range(3):
            for j in range(0, len(points)):
                if(pts[i][0] == points[j][0] and pts[i][1] == points[j][1]):
                    index.append(j)
        # Three points form a triangle
        if len(index) == 3:
            delaunayTri_indexes.append((index[0], index[1], index[2]))

    return delaunayTri_indexes, img

In [6]:
image = 'images/original_images/donald_trump_meme.jpg'
img = cv2.imread(image)
img.shape

(788, 771, 3)

In [7]:
rect = (0, 0, img.shape[1], img.shape[0])
subdiv = cv2.Subdiv2D(rect)
subdiv.insert(landmark_points)
traingle_list = subdiv.getTriangleList()
traingle_list.shape

(111, 6)

In [8]:
t = traingle_list[0]
t
pt1 = (t[0], t[1])
pt2 = (t[2], t[3])
pt3 = (t[4], t[5])

pts = [pt1, pt2, pt3]
pts

[(172.0, 202.0), (229.0, 217.0), (212.0, 233.0)]

In [9]:
pts = []
pts.append(pt1)
pts.append(pt2)
pts.append(pt3)
pts

[(172.0, 202.0), (229.0, 217.0), (212.0, 233.0)]

In [10]:
cv2.getAffineTransform?

In [25]:
hullIndex = cv2.convexHull(np.array(landmark_points), returnPoints = False)
hullIndex

array([[15],
       [14],
       [13],
       [12],
       [11],
       [10],
       [ 9],
       [ 8],
       [ 7],
       [ 6],
       [ 5],
       [ 4],
       [ 3],
       [ 2],
       [ 1],
       [ 0],
       [17],
       [18],
       [19],
       [24],
       [25],
       [26],
       [16]], dtype=int32)

In [27]:
hullIndex.shape

(23, 1)

In [29]:
def applyConvexHull(points):
    
    hullIndex = cv2.convexHull(np.array(points), returnPoints = False)
    
    hull = []
    for i in range(len(hullIndex)):
        hull.append(landmark_points[int(hullIndex[i])])
    
    return hull

In [45]:
image1 = 'images/original_images/donald_trump.jpg'
image2 = 'images/original_images/baby.jpg'

img1 = cv2.imread(image1)
img2 = cv2.imread(image2)

In [46]:
landmark_points1, _ = landmark_detection(image1)
landmark_points2, _ = landmark_detection(image2)

In [47]:
hull1 = applyConvexHull(landmark_points1)
hull2 = applyConvexHull(landmark_points2)

In [48]:
len(hull1)

22

In [49]:
len(hull2)

22

In [51]:
img1warped = np.array(img2)

In [65]:
print(img1warped.shape)
print(img1warped.dtype)
type(img1warped)

(479, 500, 3)
uint8


numpy.ndarray

In [66]:
print(img2.shape)
print(img2.dtype)
type(img2)

(479, 500, 3)
uint8


numpy.ndarray

In [70]:
a = np.array([img2.shape])

In [71]:
a

array([[479, 500,   3]])

In [72]:
a.shape

(1, 3)

In [73]:
b = a
b

array([[479, 500,   3]])

In [74]:
b.shape

(1, 3)

In [76]:
b = np.copy(a)
b

array([[479, 500,   3]])

In [77]:
b.shape

(1, 3)

In [78]:
x = np.array([1, 2, 3])
y = x
z = np.copy(x)

In [82]:
x[0] = 10
x[0] == y[0]
x


array([10,  2,  3])

In [83]:
x[0] == z[0]
z

array([1, 2, 3])